In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import scipy as sc
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
import keras

In [2]:
df = pd.read_csv('flukaRunsTyped.csv')

In [3]:
trainRatio = 0.8
# x is the tracklength in each bin
# y is the log of the energy
xfull = df.to_numpy()[:,:500]
yfull = np.log(df.to_numpy()[:,500])
particle_labels = df.to_numpy()[:,501:507]
xtrain, xtest, ytrain, ytest  = train_test_split(
    xfull, yfull, train_size=trainRatio, 
    stratify=particle_labels, shuffle=True)

In [4]:
in_size1 = xtrain.shape[1]
hiddenSize1 = 300
numEpochs1 = 100
batchSize1 = 256

lInput1 = keras.Input(shape=(in_size1,))
lEncoded1 = layers.Dense(hiddenSize1, activation='sigmoid',name='Encoder1')(lInput1)
lDecoded1 = layers.Dense(in_size1, activation='sigmoid', name = 'Decoder1')(lEncoded1)

# Full autoencoder
autoencoder1 = keras.Model(lInput1, lDecoded1)
# Only encoding part
encoder1 = keras.Model(lInput1, lEncoded1)
autoencoder1.compile(optimizer='adam', loss='MeanSquaredError')
# Run unsupervised training on the training set. We run the training for 100 epochs.
autoencoder1.fit(xtrain, xtrain, epochs=numEpochs1, batch_size=batchSize1, shuffle=True)

Epoch 1/100
188/188 [==============================] - 1s 3ms/step - loss: 0.0094
Epoch 2/100
188/188 [==============================] - 1s 3ms/step - loss: 2.1489e-04
Epoch 3/100
188/188 [==============================] - 1s 3ms/step - loss: 9.9300e-05
Epoch 4/100
188/188 [==============================] - 1s 3ms/step - loss: 5.8224e-05
Epoch 5/100
188/188 [==============================] - 1s 3ms/step - loss: 3.8576e-05
Epoch 6/100
188/188 [==============================] - 1s 3ms/step - loss: 2.7533e-05
Epoch 7/100
188/188 [==============================] - 1s 3ms/step - loss: 2.0664e-05
Epoch 8/100
188/188 [==============================] - 1s 3ms/step - loss: 1.6082e-05
Epoch 9/100
188/188 [==============================] - 1s 3ms/step - loss: 1.2865e-05
Epoch 10/100
188/188 [==============================] - 1s 3ms/step - loss: 1.0517e-05
Epoch 11/100
188/188 [==============================] - 1s 3ms/step - loss: 8.7496e-06
Epoch 12/100
188/188 [==============================] - 

In [11]:
input2 = encoder1.predict(xtrain)
xtrain2 = autoencoder1.predict(xtrain)

1500/1500 [==============================] - 1s 545us/step


In [6]:
in_size2 = input2.shape[1]
hiddenSize2 = 300
numEpochs2 = 100
batchSize2 = 256

lInput2 = keras.Input(shape=(in_size2,))
lEncoded2 = layers.Dense(hiddenSize2, activation='sigmoid',name='Encoder2')(lInput2)
lDecoded2 = layers.Dense(in_size2, activation='sigmoid', name = 'Decoder2')(lEncoded2)

# Full autoencoder
autoencoder2 = keras.Model(lInput2, lDecoded2)
# Only encoding part
encoder2 = keras.Model(lInput2, lEncoded2)
autoencoder2.compile(optimizer='adam', loss='MeanSquaredError')
# Run unsupervised training on the training set. We run the training for 100 epochs.
autoencoder2.fit(input2, input2, epochs=numEpochs2, batch_size=batchSize2, 
                 shuffle=True)

Epoch 1/100
188/188 [==============================] - 0s 2ms/step - loss: 3.3549e-04
Epoch 2/100
188/188 [==============================] - 0s 2ms/step - loss: 4.1791e-07
Epoch 3/100
188/188 [==============================] - 1s 3ms/step - loss: 4.1841e-07
Epoch 4/100
188/188 [==============================] - 0s 2ms/step - loss: 4.1999e-07
Epoch 5/100
188/188 [==============================] - 0s 2ms/step - loss: 4.2138e-07
Epoch 6/100
188/188 [==============================] - 0s 2ms/step - loss: 4.2016e-07
Epoch 7/100
188/188 [==============================] - 0s 2ms/step - loss: 4.2029e-07
Epoch 8/100
188/188 [==============================] - 0s 2ms/step - loss: 4.2175e-07
Epoch 9/100
188/188 [==============================] - 0s 2ms/step - loss: 4.1955e-07
Epoch 10/100
188/188 [==============================] - 0s 2ms/step - loss: 4.2543e-07
Epoch 11/100
188/188 [==============================] - 0s 2ms/step - loss: 4.2134e-07
Epoch 12/100
188/188 [==============================

In [7]:
input3 = encoder2.predict(input2)

1500/1500 [==============================] - 0s 309us/step


In [8]:
in_sizeFinal = input3.shape[1]
numEpochsFinal = 100
batchSizeFinal = 400

lInputFinal = keras.Input(shape=(in_sizeFinal,))
lFinal1 = layers.Dense(100, activation='sigmoid',name='Final1')(lInputFinal)
lFinal2 = layers.Dense(70, activation='sigmoid',name='Final2')(lFinal1)
lFinal3 = layers.Dense(40, activation='sigmoid',name='Final3')(lFinal2)
lFinal4 = layers.Dense(10, activation='sigmoid',name='Final4')(lFinal3)
lFinal = layers.Dense(1, activation=None,name='Final')(lFinal4)
# Full autoencoder
finalModel = keras.Model(lInputFinal, lFinal)
opt = keras.optimizers.legacy.Adam(learning_rate=0.01)
finalModel.compile(optimizer=opt, loss='MeanSquaredError')
finalModel.fit(input3, ytrain, epochs=numEpochsFinal, batch_size=batchSizeFinal, 
                 shuffle=True)

Epoch 1/100
120/120 [==============================] - 0s 1ms/step - loss: 6.5888
Epoch 2/100
120/120 [==============================] - 0s 1ms/step - loss: 4.9187
Epoch 3/100
120/120 [==============================] - 0s 1ms/step - loss: 4.9189
Epoch 4/100
120/120 [==============================] - 0s 1ms/step - loss: 4.9187
Epoch 5/100
120/120 [==============================] - 0s 1ms/step - loss: 4.9192
Epoch 6/100
120/120 [==============================] - 0s 2ms/step - loss: 4.9209
Epoch 7/100
120/120 [==============================] - 0s 2ms/step - loss: 4.9191
Epoch 8/100
120/120 [==============================] - 0s 2ms/step - loss: 4.9199
Epoch 9/100
120/120 [==============================] - 0s 2ms/step - loss: 4.9196
Epoch 10/100
120/120 [==============================] - 0s 1ms/step - loss: 4.9191
Epoch 11/100
120/120 [==============================] - 0s 1ms/step - loss: 4.9203
Epoch 12/100
120/120 [==============================] - 0s 1ms/step - loss: 4.9211
Epoch 13/100


In [16]:
logEpred = finalModel.predict(input3).reshape(ytrain.shape)

1500/1500 [==============================] - 1s 915us/step


In [19]:
np.sum((ytrain - logEpred)**2)/ytrain.shape[0]

4.9059019594433

In [21]:
#Current model predicts all energy values to be the same
np.e**logEpred

array([413.80496, 413.80496, 413.80496, ..., 413.80496, 413.80496,
       413.80496], dtype=float32)